# Problem statement and Goal
NHPD: For all counties, aggregate the National Housing Preservation Database at the census tract level using whatever variables seem most interesting or relevant, and combine it with the “processed” datasets. What does the overall low-income housing stock look like in areas with high eviction rates? Are any of these features statistically related to the incidence of evictions in these counties? Furthermore, are there any insights common to two or more counties, or is the “state” of low-income housing unique to each county?


In [1]:
import pandas as pd

In [3]:
data_set = pd.read_excel('nhpd_data.xlsx', engine='openpyxl')

### General inference from the data dictionary
- The data set 'Active and Inconclusive Properties.xlsx' appears to be the `Data Extract`, as opposed to the `Data Grid`, as each row here represents a property. Here, subsidy information is present alongside property information instead of it being expanded from the property record.
- One property - One address mapping.
- Most of the phased properties are separated by address locations. 
- This is a rare scenario: In cases where they're combined into one record, we might need to separate them by address locations. This is rare, and might not be needed since funding is tracked at property level, and unless we need specifics that sub-categorize the property, we find little use of maintaining these records as separate entities. 
- Key words 'Development', 'Project' and 'Property' have been used interchangeably to mean 'cluster of buildings' tracked by the same identification number. 
  - If we wish to run some NLP algorithms on the descriptions, we may need to replace these words as 'Property'. If not, the vectorized versions of these words may not be close to each other. 
  - Even if we consider state-of-the-art algorithms like Word2Vec, we may not get vectors close to one another because 'Development', 'Project' and 'Property' have different semantic meanings.

In [15]:
shape = data_set.shape
data_set.head()

,NHPDPropertyID,PropertyName,PropertyAddress,City,State,Zip,CBSACode,CBSAType,County,CountyCode,...,NumberActiveMR,NumberInconclusiveMR,NumberInactiveMR,Mr_1_Status,Mr_1_ProgramName,Mr_1_AssistedUnits,Mr_2_Status,Mr_2_ProgramName,Mr_2_AssistedUnits,OldNHPDPropertyID
0,1000000,IVY ESTATES,6729 Zeigler Blvd,Mobile,AL,36608-4253,33660.0,Metropolitan Statistical Area,Mobile,1097.0,...,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1000001,RENDU TERRACE WEST,7400 Old Shell Rd,Mobile,AL,36608-4549,33660.0,Metropolitan Statistical Area,Mobile,1097.0,...,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1000002,TWB RESIDENTIAL OPPORTUNITIES II,93 Canal Rd,Port Jefferson Station,NY,11776-3024,35620.0,Metropolitan,Suffolk,36103.0,...,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1000003,THE DAISY HOUSE,615 Clarissa St,Rochester,NY,14608-2485,40380.0,Metropolitan,Monroe,36055.0,...,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1000004,MAIN AVENUE APARTMENTS,105 E Walnut St,Sylacauga,AL,35150-3012,45180.0,Micropolitan Statistical Area,Talladega,1121.0,...,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
print("No. of data points:", shape[0])
print("No. of features: ", shape[1])
print("Different property statuses, :", data_set.loc[:,"PropertyStatus"].unique())

No. of data points: 82287
No. of features:  252
Different classes, : ['Active' 'Inconclusive']


In [19]:
data_set.describe()

,NHPDPropertyID,CBSACode,CountyCode,CensusTract,Latitude,Longitude,ActiveSubsidies,TotalInconclusiveSubsidies,TotalInactiveSubsidies,TotalUnits,...,NumberInconclusivePBV,NumberInactivePBV,Pbv_1_AssistedUnits,Pbv_2_AssistedUnits,NumberActiveMR,NumberInconclusiveMR,NumberInactiveMR,Mr_1_AssistedUnits,Mr_2_AssistedUnits,OldNHPDPropertyID
count,8.228700e+04,72919.000000,82229.000000,8.222400e+04,82287.000000,82287.000000,82287.000000,82287.000000,82287.000000,82287.000000,...,82287.0,82287.0,2784.000000,173.000000,82287.000000,82287.0,82287.0,500.000000,9.000000,58370.000000
mean,1.074656e+06,30447.401281,28953.566759,2.895217e+10,38.483402,-90.228069,1.386355,0.077485,0.369098,66.711145,...,0.0,0.0,43.463003,35.011561,0.006234,0.0,0.0,34.208000,26.333333,52459.141083
std,4.021620e+04,11096.935896,15256.967835,1.525466e+10,4.975471,15.636717,0.895868,0.287468,0.734841,96.200003,...,0.0,0.0,41.703263,29.062280,0.081741,0.0,0.0,25.840049,19.685020,33880.224028
min,1.000000e+06,10100.000000,1001.000000,1.001020e+09,13.495030,-166.722478,0.000000,0.000000,0.000000,1.000000,...,0.0,0.0,11.000000,11.000000,0.000000,0.0,0.0,11.000000,12.000000,4.000000
25%,1.039279e+06,19740.000000,17053.000000,1.705396e+10,34.983064,-96.380764,1.000000,0.000000,0.000000,18.000000,...,0.0,0.0,17.000000,14.000000,0.000000,0.0,0.0,16.000000,13.000000,24226.250000
50%,1.073499e+06,32580.000000,29095.000000,2.909501e+10,39.312214,-86.490946,1.000000,0.000000,0.000000,40.000000,...,0.0,0.0,29.000000,24.000000,0.000000,0.0,0.0,25.000000,19.000000,49850.500000
75%,1.108144e+06,39300.000000,41015.000000,4.101395e+10,41.799999,-79.052250,2.000000,0.000000,1.000000,82.000000,...,0.0,0.0,53.000000,46.000000,0.000000,0.0,0.0,43.000000,22.000000,78774.750000
max,1.163400e+06,99999.000000,69120.000000,5.604595e+10,65.160556,145.751129,106.000000,13.000000,24.000000,5881.000000,...,0.0,0.0,449.000000,191.000000,5.000000,0.0,0.0,187.000000,62.000000,127185.000000


### Status [WIP]
- Findout how housing and subsidies work in the US.
- Map census track to its geographical boundaries.
- List out all of the different subsidies ().
- Find out metrics per subsidy and compare various statistical plots.